# Submodule 1: Preprocessing and Quality Control

<img src="images/LessonPlan.jpg" alt="Drawing" width=1000 />

## Overview & Purpose

### Overview Video 1 - Methodology and Example Data

<video controls width=500 src="videos/ChromOccupancyVid1_wsubtitles.mp4">animation </video>

### Overview Video 2 - How to Use these Lessons

<video controls width=500 src="videos/ChromOccupancyVid2_wsubtitles.mp4">animation </video>

## ChIP-seq, CUT&RUN, and CUT&Tag
This short tutorial demonstrates the initial processing steps for ChIP-seq, CUT&RUN, and CUT&Tag analysis. In this first module, we focus on generating quality reports of the raw sequences, adapter trimming, mapping, and removal of PCR duplicates. 

<img src="images/all3methods.jpg" alt="Drawing" width=500 />

To demonstrate the process, this tutorial will analyze published datasets studying H3K27ac occupancy after aux-mediated degradation of BAF (ChIP-seq), as well as BAF (CUT&RUN) and RNA Polymerase II Ser5ph (CUT&Tag) occupancy after inhibition of transcription elongation using Flavopiridol. As such, this module covers the processing of the data from three distinct but similar methods using downsampled data to improve runtime speed. The original data can be found in the GEO repository with the following accessions: ChIP-seq - GSE173550; CUT&RUN & CUT&Tag - GSE224292. These data were published in the following articles:

Weber CM, et al. mSWI/SNF promotes Polycomb repression both directly and through genome-wide redistribution. Nat Struct Mol Biol. 2021  PMID: [34117481](https://pubmed.ncbi.nlm.nih.gov/34117481/)

Brahma S, Henikoff S. The BAF chromatin remodeler synergizes with RNA polymerase II and transcription factors to evict nucleosomes. Nat Genet. 2024 PMID: [38049663](https://pubmed.ncbi.nlm.nih.gov/38049663/)

Note that to allow faster processing we have limited the reads to that of a single chromosome (chr4).



While ChIP-seq, CUT&RUN, and CUT&Tag are all used to identify chromatin binding sites genome-wide, they differ in implementation which can impact the analysis.
<img src="images/methodcompare.png" alt="Drawing" style="width: 800px;"/>

Image credit: Kaya-Okur et al., 2020. Efficient low-cost chromatin profiling with CUT&Tag. Nature Protocols.

## ChIP-seq
This method uses random fragmentation followed by immunoprecipitation.

<img src="images/chipseq.gif" alt="Drawing" style="width: 500px;" align="center"/>

## CUT&RUN
This method uses random pA-MNase to target the fragmentation to the site where your protein is bound.

<img src="images/CUTRUN.gif" alt="Drawing" style="width: 500px;" align="center"/>

## CUT&Tag
This method uses random pA-Tn5 to directly insert sequencing adapters next to where your protein is bound.

<img src="images/CUTTag.gif" alt="Drawing" style="width: 500px;" align="center"/>


### Ways to use this module
Throughout this module, we have color-coded commands according to ChIP-seq, CUT&RUN, and CUT&Tag. Therefore this module can be used to learn about the processing of each method individually, to compare each method to the others, or you can follow the colored commands to only process one type, either ChIP-seq, CUT&RUN, or CUT&Tag.
Commands for each method will be designated by an individual logo before the command, just like the following examples

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [1]:
#run this cell for ChIP-seq
print("Code for ChIP-seq will be placed after the above image. Run these cells if performing ChIP-seq analysis.")

Code for ChIP-seq will be placed after the above image. Run these cells if performing ChIP-seq analysis.


<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [2]:
#run this cell for CUT&RUN
print("Code for CUT&RUN will be placed after the above image. Run these cells if performing CUT&RUN analysis.")

Code for CUT&RUN will be placed after the above image. Run these cells if performing CUT&RUN analysis.


<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [3]:
#run this cell for CUT&Tag
print("Code for CUT&Tag will be placed after the above image. Run these cells if performing CUT&Tag analysis.")

Code for CUT&Tag will be placed after the above image. Run these cells if performing CUT&Tag analysis.


<div class="alert alert-block alert-success" style="font-size:100%">
<span style="color:black"> By following the colors/images, you can run one, two, or all three types of analyses.</span>
</div>

### Required Files
In this stage of the module, you will use the fastq files that have been prepared. You can also use this module on your own data or any published ChIP-seq, CUT&RUN, or CUT&Tag dataset. 

<div class="alert-info" style="font-size:200%">
STEP 1: Set Up Environment
</div>

Initial items to configure your Cloud environment. In this step we will use conda to install the following packages:

Quality Reporting:
[fastqc](https://anaconda.org/bioconda/fastqc), [multiqc](https://anaconda.org/bioconda/multiqc)

Read Trimming: 
[trimmomatic](https://anaconda.org/bioconda/trimmomatic), [cutadapt](https://anaconda.org/bioconda/cutadapt)

Mapping:
[bowtie2](https://anaconda.org/bioconda/bowtie2)

In [4]:
#First let's install mamba to configure our environment
! curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
! bash Miniforge3-$(uname)-$(uname -m).sh -u -b -p $HOME/mambaforge
print("done")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 74.3M  100 74.3M    0     0   117M      0 --:--:-- --:--:-- --:--:--  207M
PREFIX=/home/ec2-user/mambaforge
Unpacking payload ...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2024.12.14-hbcca054_0.conda
Extracting ld_impl_linux-64-2.43-h712a8e2_2.conda
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting python_abi-3.12-5_cp312.conda
Extracting tzdata-2025a-h78e105d_0.conda
Extracting libgomp-14.2.0-h77fa898_1.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-14.2.0-h77fa898_1.conda
Extracting c-ares-1.34.4-hb9d3cd8_0.conda
Extracting libexpat-2.6.4-h5888daf_0.conda
Extracting libgcc-ng-14.2.0-h69a702a_1.conda
Extracting 

🕘 In the next cell we'll install several packages and configure your environment. This may take about a minute or two. 


In [5]:
#Now let's install several required packages. You will likely see "warning libmamba" messages. That is okay, please ignore these.
!mamba install -c bioconda fastqc bowtie2 multiqc trimmomatic cutadapt -y -q
!pip install --upgrade --force-reinstall jupyterquiz jupytercards
print("done")

  Using cached jupyterquiz-2.8.1-py2.py3-none-any.whl.metadata (13 kB)
  Using cached jupytercards-3.0.5-py2.py3-none-any.whl.metadata (6.5 kB)
Using cached jupyterquiz-2.8.1-py2.py3-none-any.whl (21 kB)
Using cached jupytercards-3.0.5-py2.py3-none-any.whl (15 kB)
  Attempting uninstall: jupyterquiz
    Found existing installation: JupyterQuiz 2.8.1
    Uninstalling JupyterQuiz-2.8.1:
      Successfully uninstalled JupyterQuiz-2.8.1
  Attempting uninstall: jupytercards
    Found existing installation: jupytercards 3.0.5
    Uninstalling jupytercards-3.0.5:
      Successfully uninstalled jupytercards-3.0.5
done


In [6]:
#Now let's import packages that we installed
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
numthreadsint = int(numthreads[0])
import sys
from jupyterquiz import display_quiz
from IPython.display import IFrame
#from IPython.display import display
from jupytercards import display_flashcards
import pandas as pd
#import modules for matching-type quiz
%cd questions
from quiz_module import run_quiz
%cd ../
import json
import ipywidgets as widgets
from IPython.display import display
import random
print("done")

/home/ec2-user/SageMaker/SandboxChromatinOccupancy/questions
/home/ec2-user/SageMaker/SandboxChromatinOccupancy
done


In [7]:
wd="~/SageMaker/SandboxChromatinOccupancy"
%cd $wd
#show which folder you are working in. 
!pwd

/home/ec2-user/SageMaker/SandboxChromatinOccupancy
/home/ec2-user/SageMaker/SandboxChromatinOccupancy


<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Match the method to how it implements chromatin fragmentation.
</div>

In [8]:
#Run for the quiz. Match the method to the method of chromatin fragmentation.
run_quiz("./questions/methodvsfragmentation.json", instant_feedback=True, shuffle_questions=True, shuffle_answers=True)

In [9]:
# These commands move into our Tutorial 1 directory and create our subdirectory structure.
!mkdir -p $wd/Submodule1/
%cd $wd/Submodule1/
!mkdir -p $wd/Submodule1/QC
!mkdir -p $wd/Submodule1/Trimmed
!mkdir -p $wd/Submodule1/Mapped

#Let's copy and extract our tutorial files
!wget https://chromatinoccupancytutorial.s3.us-east-2.amazonaws.com/Submodule1.2.zip
!unzip Submodule1.2.zip
print("done")

/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1
--2025-02-18 15:44:23--  https://chromatinoccupancytutorial.s3.us-east-2.amazonaws.com/Submodule1.2.zip
Resolving chromatinoccupancytutorial.s3.us-east-2.amazonaws.com (chromatinoccupancytutorial.s3.us-east-2.amazonaws.com)... 52.219.141.74, 3.5.128.58, 52.219.110.10, ...
Connecting to chromatinoccupancytutorial.s3.us-east-2.amazonaws.com (chromatinoccupancytutorial.s3.us-east-2.amazonaws.com)|52.219.141.74|:443... connected.
200 OKequest sent, awaiting response... 
Length: 366387535 (349M) [application/zip]
Saving to: ‘Submodule1.2.zip’

100%[======================================>] 366,387,535 96.6MB/s   in 3.6s   

2025-02-18 15:44:27 (96.6 MB/s) - ‘Submodule1.2.zip’ saved [366387535/366387535]

Archive:  Submodule1.2.zip
   creating: Submodule1.2/ChIPseqFiles/
  inflating: Submodule1.2/ChIPseqFiles/H3K27ac_ChIPseq_aux.fastq.gz  
  inflating: Submodule1.2/ChIPseqFiles/H3K27ac_ChIPseq_noaux.fastq.gz  
   creating: Submodul

In [10]:
#show which folder you are working in. It should now say Submodule1 at the end.
!pwd
#list the files/folders in the current directory. You should see a folder for QC, Trimmed, Mapped, and Submodule1.2.
!ls $wd/Submodule1

/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1
Mapped	QC  Submodule1.2  Submodule1.2.zip  Trimmed


<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 100px;" align="left" /> Run the following command for ChIP-seq, which lists the fastq files we will use. You should see H3K27ac_ChIPseq_aux.fastq.gz and H3K27ac_ChIPseq_noaux.fastq.gz which correspond to compressed fastq files for H3K27ac with and without auxin treatment. There is only one file for each because this is single-end data.

In [11]:
#list the files in ChIPseqFiles which we just downloaded.
!ls $wd/Submodule1/Submodule1.2/ChIPseqFiles

H3K27ac_ChIPseq_aux.fastq.gz  H3K27ac_ChIPseq_noaux.fastq.gz


<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 120px;" align="left"/> Run the following command for CUT&RUN, which lists the fastq files we will use. You should see 6 files including two for BRG1 in control, two for BRG1 after flavopiridol treatment, and two for IgG. There are two for each because we will demonstrate how to process paired-end sequence data. 

In [12]:
#list the files in CUT&RUNFiles which we just downloaded.
!ls $wd/Submodule1/Submodule1.2/CUTnRUNFiles

BRG1_con_1.fastq.gz  BRG1_FLV_1.fastq.gz  IgG_CnR_1.fastq.gz
BRG1_con_2.fastq.gz  BRG1_FLV_2.fastq.gz  IgG_CnR_2.fastq.gz


<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 120px;" align="left"/>  Run the following command for CUT&Tag, which lists the fastq files we will use. You should see 6 files including two for RNAPII-S5P in control, two for RNAPII-S5P after flavopiridol treatment, and two for IgG. There are two for each because we will demonstrate how to process paired-end sequence data. 

In [13]:
#list the files in CUT&TagFiles which we just downloaded.
!ls $wd/Submodule1/Submodule1.2/CUTnTagFiles

IgG_CnT_1.fastq.gz  RNAPII-S5P_con_1.fastq.gz  RNAPII-S5P_FLV_1.fastq.gz
IgG_CnT_2.fastq.gz  RNAPII-S5P_con_2.fastq.gz  RNAPII-S5P_FLV_2.fastq.gz


### Single-end v.s. Paired-end Sequencing

Illumina offers both single-end and paired-end sequencing. Single-end provides a short sequence from one end of the fragments. Paired-end provides a short sequence from both ends. While Single-end can be sufficient for many applications where we need a count of reads at specific positions, paired-end sequencing provides several advantages. One advantage is that paired-end reads provide more sequenced portions, useful for NGS applications that need to know the exact sequence; e.g. SNP detection and WGBS. Knowing the sequence of both ends can also help with the alignment of fragments when one end lies in a repetitive region (see the figure on the left). In this module, we'll demonstrate how to process single-end data for ChIP-seq and demonstrate processing paired-end data for CUT&RUN and CUT&Tag. However, we do recommend using paired-end sequencing for all of these methods if you are planning an experiment.

<img src="images/singleVSpairedend.gif" alt="Drawing" style="width: 500px;" align="center"/>

<div class="alert-info" style="font-size:200%">
STEP 2: QC
</div>

Sequences are typically provided as files in fastq format. This format includes four lines per sequence. 

<img src="images/fastqformat.jpg" alt="Drawing" style="width: 500px;" align="center"/>

The 3rd line is a place-holder maintained to stay consistent with earlier formats


## Click on the above image to see what each line represents.
### Next, let's take a look at the sequence quality of the raw reads using fastqc:

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 100px;" align="left"/>

In [15]:
# This command runs fastqc on each fastq.gz file inside our InputFiles directory and stores the ouput reports in our QC directory.
!fastqc -t $numthreadsint -q -o $wd/Submodule1/QC $wd/Submodule1/Submodule1.2/ChIPseqFiles/*fastq.gz

application/gzip
application/gzip


In [16]:
# We can display the resulting fastqc results.
%cd $wd
IFrame(src='Submodule1/QC/H3K27ac_ChIPseq_noaux_fastqc.html', width=1080, height=800)

/home/ec2-user/SageMaker/SandboxChromatinOccupancy


<div class="alert-success" style="font-size:100%">
After you've browsed the above report, try to visualize the quality of the other sample(s). Use the cell below to adapt the commands to visualize the bafi sample.
</div>

<div class="alert-warning" style="font-size:150%; color:black">
⬇️ Try it yourself! Within the following block, try to type out the command for fastqc on the other sample ⬇️
</div>

In [ ]:
#Type the code to visualize the fastqc report on the other sample: Submodule1/QC/H3K72ac_ChIPseq_aux_fastqc.html



Run the following cells for CUT&RUN. 🕘 The fastqc step may take a few minutes.

<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 120px;" align="left"/> 

In [17]:
#run this cell for CUT&RUN
# This command runs fastqc on each fastq.gz file inside our InputFiles directory and stores the ouput reports in our QC directory.
!fastqc -t $numthreadsint -q -o $wd/Submodule1/QC $wd/Submodule1/Submodule1.2/CUTnRUNFiles/*fastq.gz

application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip


In [18]:
# We can see an example of the resulting fastqc results.
%cd $wd
IFrame(src='Submodule1/QC/BRG1_con_1_fastqc.html', width=1080, height=800)

/home/ec2-user/SageMaker/SandboxChromatinOccupancy


<div class="alert-warning" style="font-size:150%; color:black">
⬇️ Try it yourself! Within the following block, try to type out the command for fastqc on the other samples ⬇️
</div>

In [19]:
#Type the code to visualize the fastqc report on the other CUT&RUN samples: e.g., Submodule1/QC/BRG1_FLV_1_fastqc.html


Run the following cells for CUT&Tag. 🕘 The fastqc step may take a few minutes.

<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 120px;" align="left"/>

In [20]:
#run this cell for CUT&Tag
# This command runs fastqc on each fastq.gz file inside our InputFiles directory and stores the ouput reports in our QC directory.
!fastqc -t $numthreadsint -q -o $wd/Submodule1/QC $wd/Submodule1/Submodule1.2/CUTnTagFiles/*fastq.gz

application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip


In [21]:
# We can see an example of the resulting fastqc results.
%cd $wd
IFrame(src='Submodule1/QC/RNAPII-S5P_con_1_fastqc.html', width=1080, height=800)

/home/ec2-user/SageMaker/SandboxChromatinOccupancy


<div class="alert-warning" style="font-size:150%; color:black">
⬇️ Try it yourself! Within the following block, try to type out the command for fastqc on the other samples ⬇️
</div>

In [22]:
#Type the code to visualize the fastqc report on the other CUT&Tag samples: e.g., Submodule1/QC/RNAPII-S5P_FLV_1_fastqc.html


🕘<div class="alert-warning" style="font-size:150%; color:black">
Note: You will have to wait until all CUT&RUN and CUT&Tag QC are finished to run the next commands. Since we have more CUT&RUN and CUT&Tag fastq files and they contain more reads than the ChIPseq files we are using as examples, this will take a few minutes to complete!
</div>

<div class="alert-info" style="font-size:200%">
Trimming
</div>
<img src="images/trimming.jpg" alt="Drawing" style="width: 250px;"/>
Now that we've viewed the quality let's trim the sequences to remove poor-quality bases and any adapter contamination. We'll demonstrate a package called Trimmomatic fo the ChIP-seq data.

## Let's use trimmomatic to prepare the ChIP-seq sequences before mapping.

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 100px;" align="left"/>

In [23]:
%cd $wd
#Run this cell if following the ChIP-seq tutorial.
# This will trim off N's as well as nextera adapters present in the ChIPseq library preparation. Placing the trimmed reads in our Trimmed folder.
!trimmomatic SE -threads $numthreadsint $wd/Submodule1/Submodule1.2/ChIPseqFiles/H3K27ac_ChIPseq_noaux.fastq.gz $wd/Submodule1/Trimmed/H3K27ac_ChIPseq_noaux_trimmed.fastq.gz ILLUMINACLIP:Submodule1/Submodule1.2/RefGenome/ChIPseqAdapters.fa:2:30:10 LEADING:3 TRAILING:3
#This next command is the same, except we'll do it for the bafi sample.
!trimmomatic SE -threads $numthreadsint $wd/Submodule1/Submodule1.2/ChIPseqFiles/H3K27ac_ChIPseq_aux.fastq.gz $wd/Submodule1/Trimmed/H3K27ac_ChIPseq_aux_trimmed.fastq.gz ILLUMINACLIP:Submodule1/Submodule1.2/RefGenome/ChIPseqAdapters.fa:2:30:10 LEADING:3 TRAILING:3

print("done")

/home/ec2-user/SageMaker/SandboxChromatinOccupancy
TrimmomaticSE: Started with arguments:
 -threads 1 /home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/ChIPseqFiles/H3K27ac_ChIPseq_noaux.fastq.gz /home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Trimmed/H3K27ac_ChIPseq_noaux_trimmed.fastq.gz ILLUMINACLIP:Submodule1/Submodule1.2/RefGenome/ChIPseqAdapters.fa:2:30:10 LEADING:3 TRAILING:3
Using Long Clipping Sequence: 'AGATCGGAAGAGCGGTTCAGCAGGAATGCCGAG'
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT'
Using Long Clipping Sequence: 'AGATCGGAAGAGCTCGTATGCCGTCTTCTGCTTG'
ILLUMINACLIP: Using 0 prefix pairs, 3 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Reads: 512000 Surviving: 511999 (100.00%) Dropped: 1 (0.00%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 1 /home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submo

<div class="alert-warning" style="font-size:150%; color:black">
Now that we've learned how to trim sequences to remove poor-quality bases and any adapter contamination, let's learn about an alternative package.  Let's try Cutadapt for the CUT&RUN and CUT&Tag samples.
</div>
Note: You can use Trimmoatic for CUT&RUN and CUT&Tag as well. We are simply using cutadapt to show another example.
</div>

## Let's use cutadapt to prepare the CUT&RUN and CUT&Tag sequences before mapping.

<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 120px;" align="left"/>

In [24]:
#Run this cell if following the CUT&RUN tutorial
# This will trim off N's as well as adapter sequences present in the CUT&RUN reads. Placing the trimmed reads in our Trimmed folder.
!cutadapt -j 0 -m 20 --nextseq-trim 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -Z -o $wd/Submodule1/Trimmed/BRG1_con_trimmed_1.fastq.gz -p $wd/Submodule1/Trimmed/BRG1_con_trimmed_2.fastq.gz $wd/Submodule1/Submodule1.2/CUTnRUNFiles/BRG1_con_1.fastq.gz $wd/Submodule1/Submodule1.2/CUTnRUNFiles/BRG1_con_2.fastq.gz >& $wd/Submodule1/Trimmed/BRG1_con.cut_report
print("done with BRG1_con")

#This next command is the same, except we'll do it for the BRG1 CUT&RUN with Flavopiridol treatment sample.
!cutadapt -j 0 -m 20 --nextseq-trim 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -Z -o $wd/Submodule1/Trimmed/BRG1_FLV_trimmed_1.fastq.gz -p $wd/Submodule1/Trimmed/BRG1_FLV_trimmed_2.fastq.gz $wd/Submodule1/Submodule1.2/CUTnRUNFiles/BRG1_FLV_1.fastq.gz $wd/Submodule1/Submodule1.2/CUTnRUNFiles/BRG1_FLV_2.fastq.gz >& $wd/Submodule1/Trimmed/BRG1_FLV.cut_report
print("done with BRG1_FLV")

#This next command is the same, except we'll do it for the CUT&RUN IgG sample.
!cutadapt -j 0 -m 20 --nextseq-trim 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -Z -o $wd/Submodule1/Trimmed/IgG_CnR_trimmed_1.fastq.gz -p $wd/Submodule1/Trimmed/IgG_CnR_trimmed_2.fastq.gz $wd/Submodule1/Submodule1.2/CUTnRUNFiles/IgG_CnR_1.fastq.gz $wd/Submodule1/Submodule1.2/CUTnRUNFiles/IgG_CnR_2.fastq.gz >& $wd/Submodule1/Trimmed/IgG_CnR.cut_report
print("done with all")

done with BRG1_con
done with BRG1_FLV
done with all


<div class="alert-info" style="font-size:130%">
Interactive Matching: Learn about the cutadapt command options, by trying to match the option to its function.
</div>

In [34]:
#Run for the quiz. Match the method to the method of chromatin fragmentation.
run_quiz("./questions/cutadaptoptions.json", instant_feedback=True, shuffle_questions=True, shuffle_answers=True)

In [25]:
# Now that you have removed adapter let's look at the trimming report.
!cat $wd/Submodule1/Trimmed/BRG1_con.cut_report

This is cutadapt 5.0 with Python 3.10.16
Command line parameters: -j 0 -m 20 --nextseq-trim 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -Z -o /home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Trimmed/BRG1_con_trimmed_1.fastq.gz -p /home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Trimmed/BRG1_con_trimmed_2.fastq.gz /home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/CUTnRUNFiles/BRG1_con_1.fastq.gz /home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/CUTnRUNFiles/BRG1_con_2.fastq.gz
Processing paired-end reads on 2 cores ...

=== Summary ===

Total read pairs processed:            510,000
  Read 1 with adapter:                  20,877 (4.1%)
  Read 2 with adapter:                  20,750 (4.1%)

== Read fate breakdown ==
Pairs that were too short:               7,450 (1.5%)
Pairs written (passing filters):       502,550 (98.5%)

Total basepairs processed:    25,500,000 bp
  Read 1:    

<div class="alert-warning" style="font-size:150%; color:black">
⬇️ Try it yourself! Within the following block, try to type out the command for trimming report on the other samples ⬇️
</div>

In [26]:
#Type the code to look at the trimming report on the other CUT&RUN samples


<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 120px;" align="left"/>

In [27]:
#Run this cell if following the CUT&Tag tutorial
# This will trim off N's as well as adapter sequences present in the CUT&Tag reads. Placing the trimmed reads in our Trimmed folder.
!cutadapt -j 0 -m 20 --nextseq-trim 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -Z -o $wd/Submodule1/Trimmed/RNAPII-S5P_con_trimmed_1.fastq.gz -p $wd/Submodule1/Trimmed/RNAPII-S5P_con_trimmed_2.fastq.gz $wd/Submodule1/Submodule1.2/CUTnTagFiles/RNAPII-S5P_con_1.fastq.gz $wd/Submodule1/Submodule1.2/CUTnTagFiles/RNAPII-S5P_con_2.fastq.gz >& $wd/Submodule1/Trimmed/RNAPII-S5P_con.cut_report
print("done with RNAPII-S5P_con")

#This next command is the same, except we'll do it for the BRG1 CUT&RUN with Flavopiridol treatment sample.
!cutadapt -j 0 -m 20 --nextseq-trim 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -Z -o $wd/Submodule1/Trimmed/RNAPII-S5P_FLV_trimmed_1.fastq.gz -p $wd/Submodule1/Trimmed/RNAPII-S5P_FLV_trimmed_2.fastq.gz $wd/Submodule1/Submodule1.2/CUTnTagFiles/RNAPII-S5P_FLV_1.fastq.gz $wd/Submodule1/Submodule1.2/CUTnTagFiles/RNAPII-S5P_FLV_2.fastq.gz >& $wd/Submodule1/Trimmed/RNAPII-S5P_FLV.cut_report
print("done with RNAPII-S5P_FLV")

#This next command is the same, except we'll do it for the CUT&RUN IgG sample.
!cutadapt -j 0 -m 20 --nextseq-trim 20 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -Z -o $wd/Submodule1/Trimmed/IgG_CnT_trimmed_1.fastq.gz -p $wd/Submodule1/Trimmed/IgG_CnT_trimmed_2.fastq.gz $wd/Submodule1/Submodule1.2/CUTnTagFiles/IgG_CnT_1.fastq.gz $wd/Submodule1/Submodule1.2/CUTnTagFiles/IgG_CnT_2.fastq.gz >& $wd/Submodule1/Trimmed/IgG_CnT.cut_report
print("done with all")

done with RNAPII-S5P_con
done with RNAPII-S5P_FLV
done with all


<div class="alert-warning" style="font-size:150%; color:black">
⬇️ Try it yourself! Within the following block, try to type out the command for trimming report on the other samples ⬇️
</div>

In [28]:
#Type the code to look at the trimming report on the other CUT&Tag samples


<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Choose one or more answers about why we trimmed the reads.
</div>

In [29]:
%cd $wd
display_quiz("questions/trimmingQuiz.json")

/home/ec2-user/SageMaker/SandboxChromatinOccupancy


<IPython.core.display.Javascript object>

<div class="alert-info" style="font-size:200%">
STEP 3: Mapping
</div>
Our fastq files include sequences and quality scores for each base, but we want to figure out which genomic location these sequences came from. To do this we will map each sequence to a reference genome using bowtie2. 

<img src="images/ReferenceGenome.png" alt="Drawing" style="width: 400px;"/>

Genome Index

Finding a matching sequence in billions of base pairs of sequence is a difficult task. Consider how much time it would take to match 20 million reads of sequence! 

To facilitate this process, alignment tools use a genome index. Think of this like a book index. The genome index is a fast way for the tool to look up specific "seeds" of sequence (seeds are a small portion of the sequence that can match multiple locations). Once it has the locations of the "seed" portion, it can then narrow down to the exact location based on the entire sequence of the read.

<img src="images/indexsearch.jpg" alt="Drawing" style="width: 300px;"/>

There are several different alignment tools, and each uses a different way of creating the index. We will use a tool called bowtie2, so we'll need indexes created by bowtie2. Because this is a long but easy process, I'm going to give you the command, but you won't be expected to run it. 

Mapping reads requires a reference genome. Due to time and memory considerations, in this tutorial we  prepared that file for you and will only map to chr4. However, in a full analysis, we would map to the entire genome. To do so you would need a fasta file corresponding to the reference genome (e.g. [mm39](https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M36/GRCm39.primary_assembly.genome.fa.gz)) from which you'd create an index of the genome using bowtie2-build. This can be done with the command: 

bowtie2-build reference_genome_file.fa outputprefix.

As mentioned, we've gone ahead and created the index for you, and, earlier, you extracted them into the RefGenome directory. These index files end in the bt2 extension. 

In [30]:
!ls $wd/Submodule1/Submodule1.2/RefGenome/*bt2

/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/RefGenome/mm39_chr4.1.bt2
/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/RefGenome/mm39_chr4.2.bt2
/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/RefGenome/mm39_chr4.3.bt2
/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/RefGenome/mm39_chr4.4.bt2
/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/RefGenome/mm39_chr4.rev.1.bt2
/home/ec2-user/SageMaker/SandboxChromatinOccupancy/Submodule1/Submodule1.2/RefGenome/mm39_chr4.rev.2.bt2


<div class="alert-success" style="font-size:100%">
These index files were created from our fasta file
 </div>

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Choose the correct answer(s).
</div>

In [38]:
%cd $wd
display_quiz("questions/bowtie2Quiz.json")

/home/ec2-user/SageMaker/SandboxChromatinOccupancy


<IPython.core.display.Javascript object>

Let's map the data to our genome build.

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 100px;" align="left"/> This may take a few minutes: 🕘

In [31]:
%cd $wd
# Notes: The -x option specifies the prefix of the index. -U specifies our fastq file (use -1 and -2 if you have paired-end reads). -S specifies our output file in sam format.
!bowtie2 -p $numthreadsint -x ./Submodule1/Submodule1.2/RefGenome/mm39_chr4 -U ./Submodule1/Trimmed/H3K27ac_ChIPseq_noaux_trimmed.fastq.gz -S ./Submodule1/Mapped/H3K27ac_ChIPseq_noaux.sam
#This runs bowtie2 again, for the other sample.
!bowtie2 -p $numthreadsint -x ./Submodule1/Submodule1.2/RefGenome/mm39_chr4 -U ./Submodule1/Trimmed/H3K27ac_ChIPseq_aux_trimmed.fastq.gz -S ./Submodule1/Mapped/H3K27ac_ChIPseq_aux.sam

/home/ec2-user/SageMaker/SandboxChromatinOccupancy
511999 reads; of these:
  511999 (100.00%) were unpaired; of these:
    18346 (3.58%) aligned 0 times
    172894 (33.77%) aligned exactly 1 time
    320759 (62.65%) aligned >1 times
96.42% overall alignment rate
511000 reads; of these:
  511000 (100.00%) were unpaired; of these:
    19340 (3.78%) aligned 0 times
    171143 (33.49%) aligned exactly 1 time
    320517 (62.72%) aligned >1 times
96.22% overall alignment rate


<div class="alert-info" style="font-size:130%">
Interactive Matching: Learn about the bowtie2 command options, by trying to match the option to its function.
</div>

In [35]:
#Run for the quiz. Match the method to the method of chromatin fragmentation.
run_quiz("./questions/bowtie2options.json", instant_feedback=True, shuffle_questions=True, shuffle_answers=True)

<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 120px;" align="left"/> This may take a few minutes: 🕘

In [32]:
%cd $wd
# Notes: The -x option specifies the prefix of the index. -1 and -2 specifies the left and righ end fastq file (using -1 and -2 because we have paired-end reads). -S specifies our output file in sam format.
!bowtie2 -p $numthreadsint -x ./Submodule1/Submodule1.2/RefGenome/mm39_chr4 -1 ./Submodule1/Trimmed/BRG1_con_trimmed_1.fastq.gz -2 ./Submodule1/Trimmed/BRG1_con_trimmed_2.fastq.gz -S ./Submodule1/Mapped/BRG1_CnR_con.sam
#This runs bowtie2 again, for the other sample.
!bowtie2 -p $numthreadsint -x ./Submodule1/Submodule1.2/RefGenome/mm39_chr4 -1 ./Submodule1/Trimmed/BRG1_FLV_trimmed_1.fastq.gz -2 ./Submodule1/Trimmed/BRG1_FLV_trimmed_2.fastq.gz -S ./Submodule1/Mapped/BRG1_CnR_FLV.sam
#This runs bowtie2 again, for the IgG sample.
!bowtie2 -p $numthreadsint -x ./Submodule1/Submodule1.2/RefGenome/mm39_chr4 -1 ./Submodule1/Trimmed/IgG_CnR_trimmed_1.fastq.gz -2 ./Submodule1/Trimmed/IgG_CnR_trimmed_2.fastq.gz -S ./Submodule1/Mapped/IgG_CnR.sam


/home/ec2-user/SageMaker/SandboxChromatinOccupancy
502550 reads; of these:
  502550 (100.00%) were paired; of these:
    489000 (97.30%) aligned concordantly 0 times
    9235 (1.84%) aligned concordantly exactly 1 time
    4315 (0.86%) aligned concordantly >1 times
    ----
    489000 pairs aligned concordantly 0 times; of these:
      116950 (23.92%) aligned discordantly 1 time
    ----
    372050 pairs aligned 0 times concordantly or discordantly; of these:
      744100 mates make up the pairs; of these:
        74264 (9.98%) aligned 0 times
        55716 (7.49%) aligned exactly 1 time
        614120 (82.53%) aligned >1 times
92.61% overall alignment rate
502493 reads; of these:
  502493 (100.00%) were paired; of these:
    489128 (97.34%) aligned concordantly 0 times
    9351 (1.86%) aligned concordantly exactly 1 time
    4014 (0.80%) aligned concordantly >1 times
    ----
    489128 pairs aligned concordantly 0 times; of these:
      116962 (23.91%) aligned discordantly 1 time
   

<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 120px;" align="left"/> This may take a few minutes: 🕘

In [33]:
%cd $wd
# Notes: The -x option specifies the prefix of the index. -1 and -2 specifies the left and righ end fastq file (using -1 and -2 because we have paired-end reads). -S specifies our output file in sam format.
!bowtie2 -p $numthreadsint -x ./Submodule1/Submodule1.2/RefGenome/mm39_chr4 -1 ./Submodule1/Trimmed/RNAPII-S5P_con_trimmed_1.fastq.gz -2 ./Submodule1/Trimmed/RNAPII-S5P_con_trimmed_2.fastq.gz -S ./Submodule1/Mapped/RNAPII-S5P_CnT_con.sam
#This runs bowtie2 again, for the other sample.
!bowtie2 -p $numthreadsint -x ./Submodule1/Submodule1.2/RefGenome/mm39_chr4 -1 ./Submodule1/Trimmed/RNAPII-S5P_FLV_trimmed_1.fastq.gz -2 ./Submodule1/Trimmed/RNAPII-S5P_FLV_trimmed_2.fastq.gz -S ./Submodule1/Mapped/RNAPII-S5P_CnT_FLV.sam
#This runs bowtie2 again, for the IgG sample.
!bowtie2 -p $numthreadsint -x ./Submodule1/Submodule1.2/RefGenome/mm39_chr4 -1 ./Submodule1/Trimmed/IgG_CnT_trimmed_1.fastq.gz -2 ./Submodule1/Trimmed/IgG_CnT_trimmed_2.fastq.gz -S ./Submodule1/Mapped/IgG_CnT.sam


/home/ec2-user/SageMaker/SandboxChromatinOccupancy
455713 reads; of these:
  455713 (100.00%) were paired; of these:
    441442 (96.87%) aligned concordantly 0 times
    7970 (1.75%) aligned concordantly exactly 1 time
    6301 (1.38%) aligned concordantly >1 times
    ----
    441442 pairs aligned concordantly 0 times; of these:
      128659 (29.15%) aligned discordantly 1 time
    ----
    312783 pairs aligned 0 times concordantly or discordantly; of these:
      625566 mates make up the pairs; of these:
        57166 (9.14%) aligned 0 times
        49801 (7.96%) aligned exactly 1 time
        518599 (82.90%) aligned >1 times
93.73% overall alignment rate
485823 reads; of these:
  485823 (100.00%) were paired; of these:
    478186 (98.43%) aligned concordantly 0 times
    3767 (0.78%) aligned concordantly exactly 1 time
    3870 (0.80%) aligned concordantly >1 times
    ----
    478186 pairs aligned concordantly 0 times; of these:
      133458 (27.91%) aligned discordantly 1 time
   

# Bowtie2 outputs a file in [sam format](https://samtools.github.io/hts-specs/SAMv1.pdf), which contains the original sequence, quality scores, and the genomic coordinates matching each read. 

<img src="images/samformat.jpg" alt="Drawing" style="width: 800px;"/>

Each read is also given a SAM flag (second column). This is a numerical code corresponding to various features of the mapping.

A flag of 0 means the read was single-end and mapped to the plus strand. A flag 16 means the read was single-end and mapped to the minus strand. A flag of 4 means the read was unmapped. These are the most common values you will see with single-end data. 

You can also look up what different SAM flag numerical codes mean: https://broadinstitute.github.io/picard/explain-flags.html

<img src="images/explainsam.jpg" alt="Drawing" style="width: 400px;"/>

### While we obtained high alignment rate, let's discuss a few features that can impact alignment. 

Remember that some of the genome is repetitive and so some of our reads may match multiple locations. Bowtie2 reports the location of each read, as well as a Map Quality score for each. This quality score is impacted by several factors, including how many loci it could match as well as the base quality (confidence in the sequence) we received. 

If you expect issues with repetitive regions (for example if your protein is abundant near pericentromeres or transposons) then using longer reads can help. For most purposes, we can get by with fairly short reads. 

<img src="images/repetitive.gif" alt="Drawing" style="width: 400px;"/>

<div class="alert alert-block alert-success" style="font-size:120%">
<span style="color:black">Congrats! You have successfully preprocessed and mapped the samples. In the next tutorial, we will continue with the processing by filtering for quality, visualizing the signal, identifying peaks, and performing differential analysis.</span>
</div>


# Continue on to [Submodule2](Submodule2.ipynb)